In [ ]:
!pip install --upgrade transformers huggingface_hub
!pip install peft==0.4.0
!pip install bitsandbytes==0.41.3
!pip install datasets
!pip install wandb
!pip install huggingface_hub
!pip install accelerate==0.26.0
!pip install trl==0.4.7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import pandas as pd
from datasets import Dataset, load_dataset

from transformers import(
AutoTokenizer,
AutoModelForCausalLM,
TrainingArguments,
Trainer
)

from peft import LoraConfig, get_peft_model, TaskType
import bitsandbytes as bnb
import os
from huggingface_hub import login
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer, default_data_collator
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
vulnerability_dataset = load_dataset("CyberNative/Code_Vulnerability_Security_DPO")
dataset = vulnerability_dataset['train']
shuffled_dataset = dataset.shuffle(seed = 42)
split_dataset = shuffled_dataset.train_test_split(test_size=0.2, seed = 42)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

secure_programming_dpo.json:   0%|          | 0.00/6.87M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4656 [00:00<?, ? examples/s]

In [ ]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

# Verify the sizes of the splits
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

Train size: 3724
Test size: 932


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    device_map="auto",
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast = False)
if not tokenizer.pad_token:
    tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

# Alternatively, add a custom padding token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Resize embeddings if needed

In [ ]:
def format_example(example):
    """
    Formats the input example into the desired structure for fine-tuning.
    """
    language = example.get('lang', 'Unknown')
    vulnerability = example.get('vulnerability', '')
    scenario = example.get('question', '')
    input_code = example.get('rejected', '')
    corrected_code = example.get('chosen', '')

    formatted_string = f"""
    ### Language:
    {language}

    ### Scenario:
    {scenario}

    ###This is my code:
    ```{language}
    {input_code}

    ### Task:
    1. Identify and describe the vulnerability in the code. Begin your answer with 'Vulnerability:'.
    2. Rewrite the program to fix the vulnerability. Begin your corrected program with 'Corrected Code:'.

    Vulnerability: {vulnerability}
    Corrected Code: {corrected_code}
    """

    return formatted_string

In [ ]:
def tokenize_function(examples):
    """
    Tokenize the formatted examples from the dataset.
    """
    # Create formatted strings for the batch
    formatted_examples = [
        format_example({
            'lang': lang,
            'vulnerability': vulnerability,
            'question': question,
            'chosen': chosen,
            'rejected': rejected,
        })
        for lang, vulnerability, question, chosen, rejected in zip(
            examples['lang'],
            examples['vulnerability'],
            examples['question'],
            examples['chosen'],
            examples['rejected'],
        )
    ]

    # Tokenize the formatted examples
    tokenized = tokenizer(
        formatted_examples,
        padding="max_length",
        truncation=True,
        max_length=1024,
    )

    # Set the labels
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized


In [ ]:
one_shot_prompt = """

### Language:
Python

### Scenario:
I am using Python to implement a program that calculates the sum of a list of numbers provided by the user. The user inputs a string of comma-separated numbers which is converted into a list and summed.

This is my code:
```python
def calculate_sum(user_input):
    numbers = [int(x) for x in user_input.split(",")]
    total = sum(numbers)
    print(f"Total sum: {total}")

user_input = input("Enter numbers separated by commas: ")
calculate_sum(user_input)

### Task:
1. Identify and describe the vulnerability in the code. Begin your answer with "Vulnerability:".
2. Rewrite the code to fix the vulnerability. Begin your corrected program with "Corrected Code:".
"""

In [ ]:
final_prompt = one_shot_prompt
input_ids = tokenizer(final_prompt, return_tensors="pt").input_ids
output = model.generate( input_ids, max_length=1024, temperature=0.7, top_p=0.95, repetition_penalty=1.2, num_return_sequences=1, )

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2134: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)



### Language:
Python

### Scenario:
I am using Python to implement a program that calculates the sum of a list of numbers provided by the user. The user inputs a string of comma-separated numbers which is converted into a list and summed.

This is my code:
```python
def calculate_sum(user_input):
    numbers = [int(x) for x in user_input.split(",")]
    total = sum(numbers)
    print(f"Total sum: {total}")

user_input = input("Enter numbers separated by commas: ")
calculate_sum(user_input)

### Task:
1. Identify and describe the vulnerability in the code. Begin your answer with "Vulnerability:".
2. Rewrite the code to fix the vulnerability. Begin your corrected program with "Corrected Code:".
3. How can you use this solution as part of an automation testing approach? Explain how it will help testers do their jobs better, faster or cheaper than before. (e.g., speed up execution time; reduce amount of data required for proper verification/validation; etc.)
4. Provide any other insights

In [ ]:
few_shot_prompt = """
### Language:
C++

### Scenario:
I am using C++ to implement a program that copies user input into a buffer. Write a function that takes user input and copies it into a character array.

This is my code:
```cpp
#include <iostream>
#include <cstring>

void copyInput(const char* userInput) {
    char buffer[10];
    strcpy(buffer, userInput);
    std::cout << "Input copied: " << buffer << std::endl;
}

int main() {
    char input[100];
    std::cout << "Enter your input: ";
    std::cin >> input;

    copyInput(input);
    return 0;
}

### Task:
1. Identify and describe the vulnerability in the code. Begin your answer with "Vulnerability:".
2. Rewrite the program to fix the vulnerability. Begin your corrected program with "Corrected Code:".

Vulnerability: Improper memory allocation and lack of boundary checks can lead to buffer overflow vulnerabilities in C++ programs.

Corrected code:

#include <iostream>
#include <cstring>

void copyInput(const char* userInput) {
    char buffer[10];
    strncpy(buffer, userInput, sizeof(buffer) - 1);
    buffer[sizeof(buffer) - 1] = '\0';
    std::cout << "Input copied: " << buffer << std::endl;
}

int main() {
    char input[100];
    std::cout << "Enter your input: ";
    std::cin >> input;

    copyInput(input);
    return 0;
}"""

new_question = """
Language:
JavaScript

Scenario:
I am using JavaScript to build a web server that dynamically executes code based on user input. Write a function that evaluates user-provided JavaScript code using eval().


This is my code:
const express = require('express');
const app = express();

app.use(express.json());

app.post('/execute', (req, res) => {
    const userCode = req.body.code;
    const result = eval(userCode);

    res.send(`Result: ${result}`);
});

app.listen(3000, () => {
    console.log('Server running on port 3000');
});

### Task:
1. Identify and describe the vulnerability in the code. Begin your answer with "Vulnerability:".
2. Rewrite the program to fix the vulnerability. Begin your corrected program with "Corrected Code:".
"""

In [ ]:
final_prompt = few_shot_prompt + new_question
input_ids = tokenizer(final_prompt, return_tensors="pt").input_ids
output = model.generate( input_ids, max_length=1024, temperature=0.7, top_p=0.95, repetition_penalty=1.2, num_return_sequences=1, )

In [ ]:
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


### Language:
C++

### Scenario:
I am using C++ to implement a program that copies user input into a buffer. Write a function that takes user input and copies it into a character array.

This is my code:
```cpp
#include <iostream>
#include <cstring>

void copyInput(const char* userInput) {
    char buffer[10];
    strcpy(buffer, userInput);
    std::cout << "Input copied: " << buffer << std::endl;
}

int main() {
    char input[100];
    std::cout << "Enter your input: ";
    std::cin >> input;

    copyInput(input);
    return 0;
}

### Task:
1. Identify and describe the vulnerability in the code. Begin your answer with "Vulnerability:".
2. Rewrite the program to fix the vulnerability. Begin your corrected program with "Corrected Code:".

Vulnerability: Improper memory allocation and lack of boundary checks can lead to buffer overflow vulnerabilities in C++ programs.

Corrected code:

#include <iostream>
#include <cstring>

void copyInput(const char* userInput) {
    char buffer[10];

In [ ]:
# Specify columns to retain
columns_to_keep = ['lang', 'vulnerability', 'question', 'chosen', 'rejected']

# Remove unnecessary columns dynamically
columns_to_remove_train = [col for col in train_dataset.column_names if col not in columns_to_keep]
columns_to_remove_test = [col for col in test_dataset.column_names if col not in columns_to_keep]

# Tokenize train dataset
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=columns_to_remove_train,
)

# Tokenize test dataset
test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=columns_to_remove_test,
)

# Verify tokenized datasets
print("Tokenized Train Dataset Sample:", train_dataset[0])
print("Tokenized Test Dataset Sample:", test_dataset[0])



Map:   0%|          | 0/3724 [00:00<?, ? examples/s]

Map:   0%|          | 0/932 [00:00<?, ? examples/s]

Tokenized Train Dataset Sample: {'lang': 'php', 'vulnerability': "In PHP, it's possible for an unsanitized user input to lead to SQL injection attacks.", 'question': "Write a php code that connects to a MySQL database named 'test' on localhost. The code should prepare a SQL statement to select all rows from the 'users' table where the 'username' and 'password' match those provided in the URL parameters. Then, the code should execute the prepared statement and print out each row.", 'chosen': '```php\n<?php\n$db = new PDO(\'mysql:host=localhost;dbname=test\', $user, $pass);\n\n$stmt = $db->prepare("SELECT * FROM users WHERE username = :username AND password = :password");\n\n$username = filter_input(INPUT_GET, \'username\', FILTER_SANITIZE_STRING);\n$password = filter_input(INPUT_GET, \'password\', FILTER_SANITIZE_STRING);\n\n$stmt->bindParam(\':username\', $username);\n$stmt->bindParam(\':password\', $password);\n\n$stmt->execute();\n\nwhile ($row = $stmt->fetch()) {\n    print_r($row);

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=32,  # Rank of the update matrices
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Target specific modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)


In [ ]:
model.print_trainable_parameters()

trainable params: 33,554,432 || all params: 3,533,967,360 || trainable%: 0.9494833591219133


In [ ]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import default_data_collator

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Models_LLM_Project/meta-llama-7b-lora-security",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,  # Increase if memory allows
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    load_best_model_at_end=False,
    report_to="wandb",  # Disable reporting to external tools like WandB
)


data_collator = default_data_collator



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
50,5.655500,0.169817
100,4.730700,0.142557
150,4.002100,0.132462
200,3.788700,0.126383
250,3.420900,0.123290
300,3.343000,0.121214


TrainOutput(global_step=348, training_loss=16.762316824375898, metrics={'train_runtime': 4247.1553, 'train_samples_per_second': 2.63, 'train_steps_per_second': 0.082, 'total_flos': 4.5273541472629555e+17, 'train_loss': 16.762316824375898, 'epoch': 2.979591836734694})

In [ ]:
# Save the LoRA adapters
model.save_pretrained("/content/drive/MyDrive/Models_LLM_Project/meta-llama-7b-lora-security")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Models_LLM_Project/meta-llama-7b-lora-security")

('/content/drive/MyDrive/Models_LLM_Project/meta-llama-7b-lora-security/tokenizer_config.json',
 '/content/drive/MyDrive/Models_LLM_Project/meta-llama-7b-lora-security/special_tokens_map.json',
 '/content/drive/MyDrive/Models_LLM_Project/meta-llama-7b-lora-security/tokenizer.model',
 '/content/drive/MyDrive/Models_LLM_Project/meta-llama-7b-lora-security/added_tokens.json')

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch
import pandas as pd
from datasets import Dataset, load_dataset

from transformers import(
AutoTokenizer,
AutoModelForCausalLM,
TrainingArguments,
Trainer,
BitsAndBytesConfig
)

from peft import LoraConfig, get_peft_model, TaskType
import bitsandbytes as bnb
# Load the base model with 4-bit quantization
# Load the model with 4-bit quantization

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    device_map="auto",
)


# Load the LoRA adapters
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/Models_LLM_Project/meta-llama-7b-lora-security")

# Set to evaluation mode
model.eval()